<a href="https://colab.research.google.com/github/uditbhaskar21/PySpark/blob/main/callduraration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=cd0fff48f1a58b87d5236857071069b5844635e2344fe630e9b89b74e319f35d
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [3]:
from pyspark.sql.functions import col,sum,when, count

In [4]:
data = [
    (10,20,58),
    (20,10,12),
    (10,30,20),
    (30,40,100),
    (30,40,200),
    (30,40,200),
    (40,30,500),
]

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("example").getOrCreate()
print(spark.version)

3.4.1


In [8]:
df = spark.createDataFrame(data,["from","to","dur"])


In [9]:
df.show()

+----+---+---+
|from| to|dur|
+----+---+---+
|  10| 20| 58|
|  20| 10| 12|
|  10| 30| 20|
|  30| 40|100|
|  30| 40|200|
|  30| 40|200|
|  40| 30|500|
+----+---+---+



PySpark

In [11]:
df1 = (df.withColumn("P1",when(col("from")< col("to"), col("from")).otherwise(col("to")))
.withColumn("P2",when(col("from")< col("to"),col("to")).otherwise(col("from"))).select("P1","P2","Dur"))

In [12]:
df1.show()

+---+---+---+
| P1| P2|Dur|
+---+---+---+
| 10| 20| 58|
| 10| 20| 12|
| 10| 30| 20|
| 30| 40|100|
| 30| 40|200|
| 30| 40|200|
| 30| 40|500|
+---+---+---+



In [15]:
result = (df1.groupBy(col("P1"),col("P2")).agg(sum(col("Dur")).alias("Duration"), count(col("P1")).alias("call_count")).select("P1","P2","Duration","call_count"))

In [16]:
result.show()

+---+---+--------+----------+
| P1| P2|Duration|call_count|
+---+---+--------+----------+
| 10| 30|      20|         1|
| 10| 20|      70|         2|
| 30| 40|    1000|         4|
+---+---+--------+----------+



SparkSQL

In [17]:
df.createOrReplaceTempView("Calls")

In [ ]:
select
case
when from < to then from else to end P1
when from < to then to else from end P2
, duration from calls


with cte1 as (
    select *, case when from < to then from else to end P1,
    case when from < to then  to else from end P2
    from calls)
)

select p1,p2, count(p1), sum(duration) from cte1
group by p1,p2
